In [191]:
import json
from functools import cmp_to_key
import functools
from inspect import getsource
from sys import argv
import re

complete_match = ["""import json as a,sys
b=sys.argv
a.dump(dict(zip(*[sorted(x)for x in a.load(open(b[1]))])),open(b[2],'w'))""",
                      """import json as a,sys,re
b=sys.argv
a.dump(dict(zip(*[sorted(i,key=lambda y:[int(c)if c.isdigit()else c for c in re.split('(\d+)',y)])for i in a.load(open(b[1]))])),open(b[2],'w'))""",
                      """import json as a,sys,re
from functools import cmp_to_key
b=sys.argv
def Z(rhs,lhs):
    return 1 if int(rhs)>int(lhs) else -1
def dimensions(string,i):
    it=list(re.finditer('{{}}'.format('\[([0-9]+)\]'*i),string))
    return (0,0) if not i else (i,it) if len(it) else dimensions(string,i-1)
def order(string):
    it=re.findall('\$O([0-9]+)',string)
    return int(it[0]) if len(it) else 0
def dictionary_order(R,L):
    return 1 if R>L else 0 if R==L else -1
def iter_bit(dim,R,L):
    for i in range(dim):
        if R[i]!=L[i]:
            return Z(R[i],L[i])
    return 0
def dimension_based(R,L):
    rd=dimensions(R[0],2)
    ld=dimensions(L[0],2)
    if rd[0]==ld[0] and rd[0]!=0:
        X,Y=order(R[0]),order(L[0])
        if R[0][:rd[1][0].start()]==L[0][:ld[1][0].start()]:
            if X-1>=len(rd[1]) and Y-1>=len(ld[1]):
                return 1 if X>Y else -1
            if X-1>=len(rd[1]) or Y-1>=len(ld[1]):
                return 1 if X-1>=len(rd[1]) else -1
            res=iter_bit(rd[0],rd[1][X-1].groups(),ld[1][Y-1].groups())
            return res if res else 1 if X > Y else -1 
    return dictionary_order(R[0],L[0])
a.dump(dict(zip(*[sorted(x,key=cmp_to_key(dimension_based))for x in a.load(open(b[1]))])),open(b[2],'w'))"""]
    
incomplete_match = ["""import json as a,sys
b=sys.argv
A=[sorted(x)for x in a.load(open(b[1]))]
t={}
B=A[1][:]
D=[]
C=0
for i in range(0,len(t),2):
    if type(t[i])!=list:
        D+=[B[t[i]+C+j]for j in range(t[i+1])]
        C+=t[i+1]
    else:
        D+=[B[t[i][j]+C+j]for j in range(len(t[i]))]
        C+=len(t[i])
A[1]=D
a.dump(dict(zip(*A)),open(b[2],'w'))""",
                        '''import json as a,sys,re
t={}
b=sys.argv
A=[sorted(i,key=lambda y:[int(c)if c.isdigit()else c for c in re.split('(\d+)',y)])for i in a.load(open(b[1]))]
B=A[1][:]
D=[]
C=0
for i in range(0,len(t),2):
    if type(t[i])!=list:
        D+=[B[t[i]+C+j]for j in range(t[i+1])]
        C+=t[i+1]
    else:
        D+=[B[t[i][j]+C+j]for j in range(len(t[i]))]
        C+=len(t[i])
A[1]=D
a.dump(dict(zip(*A)),open(b[2],'w'))''',
                         """import json as a,sys,re
from functools import cmp_to_key
b=sys.argv
def Z(rhs,lhs):
    return 1 if int(rhs)>int(lhs) else -1
def dimensions(string,i):
    it=list(re.finditer('{{}}'.format('\[([0-9]+)\]'*i),string))
    return (0,0) if not i else (i,it) if len(it) else dimensions(string,i-1)
def order(string):
    it=re.findall('\$O([0-9]+)',string)
    return int(it[0]) if len(it) else 0
def dictionary_order(R,L):
    return 1 if R>L else 0 if R==L else -1
def iter_bit(dim,R,L):
    for i in range(dim):
        if R[i]!=L[i]:
            return Z(R[i],L[i])
    return 0
def dimension_based(R,L):
    rd=dimensions(R[0],2)
    ld=dimensions(L[0],2)
    if rd[0]==ld[0] and rd[0]!=0:
        X,Y=order(R[0]),order(L[0])
        if R[0][:rd[1][0].start()]==L[0][:ld[1][0].start()]:
            if X-1>=len(rd[1]) and Y-1>=len(ld[1]):
                return 1 if X>Y else -1
            if X-1>=len(rd[1]) or Y-1>=len(ld[1]):
                return 1 if X-1>=len(rd[1]) else -1
            res=iter_bit(rd[0],rd[1][X-1].groups(),ld[1][Y-1].groups())
            return res if res else 1 if X > Y else -1 
    return dictionary_order(R[0],L[0])
A=[sorted(x,key=cmp_to_key(dimension_based))for x in a.load(open(b[1]))]
t={}
B=A[1][:]
D=[]
C=0
for i in range(0,len(t),2):
    if type(t[i])!=list:
        D+=[B[t[i]+C+j]for j in range(t[i+1])]
        C+=t[i+1]
    else:
        D+=[B[t[i][j]+C+j]for j in range(len(t[i]))]
        C+=len(t[i])
A[1]=D
a.dump(dict(zip(*A)),open(b[2],'w'))"""]

def shift_count(ori,new):
    count = 0
    for k,v in ori.items():
        if new[k] != v:
            count += 1
    return count

def generate_shift_table(ori,new):
    count = 0
    idx=[]
    for i in range(len(ori)):
        if ori[new[0][i]] != new[1][i]:
            count += 1
            idx.append(new[1].index(ori[new[0][i]])-i)
        else:
            idx.append(0)
    return idx

def generate_run_length_table(idx):
    # [initialization]
    # add end flag
    idx += ['*']  
    # current run
    current = idx[0] 
    # current length
    run_length = 0
    # final output info
    res = []
    # readble output info
    readable = []
    
    # [run_length]
    for i in range(len(idx)):
        if idx[i] == current:
            run_length += 1
        else:
            res += [current,run_length]
            readable.append([current,run_length])
            current = idx[i]
            run_length = 1
    return res, readable

def two_level_run_length(table):
    flag = False
    temp = []
    tltable = []
    for i in range(0,len(table),2):
        if not flag and table[i+1] == 1:
            temp = [[table[i]],1]
            flag = True
        elif flag and table[i+1] == 1:
            temp[0] += [table[i]]
        else:
            flag = False
            tltable += temp
            tltable += table[i:i+2]
            temp = []
    tltable += temp
    return tltable

def generate_tables(origin_table, input_table, method):
    methods = ['Default', 'Natural', 'Order']
    choise, idx, res, readable, tltable = -1, [], [], [], []
    ns = shift_count(origin_table, input_table)
    if ns == 0:
        print('%s key can sovle this case.' % method)
        choise = methods.index(method)
    else:
        print('%s key can\'t sovle this case.' % method)
        print('Shift count in this method: %d.' % ns)
        idx = generate_shift_table(origin_table,[list(input_table.keys()),list(input_table.values())])
        res, readable = generate_run_length_table(idx)
        tltable = two_level_run_length(res)
        print('Shift table string length: %d.' % len(str(idx)))
        print('One level run-length table string length: %d.' % len("".join(str(res).split(' '))))
        print('The most large consecutive segment length: %d.' % max([*zip(*readable)][1]))
        print('Two level run-length table string length: %d.' % len("".join(str(tltable).split(' '))))
    return choise, idx, res, readable, tltable

def default_key_method(origin_mapping):
    keys, values = zip(*origin_mapping.items())
    keys, values = sorted(keys), sorted(values)
    return dict(zip(keys,values))

def natural_key(x):
    return [int(c)if c.isdigit()else c for c in re.split('(\d+)',x)]

def natural_key_method(origin_mapping):
    ori = [list(origin_mapping.keys()),list(origin_mapping.values())]
    return dict(zip(*map(lambda x:sorted(x,key=natural_key),ori)))

def A(rhs,lhs):
    return 1 if int(rhs)>int(lhs) else -1
def dimensions(string,i):
    it=list(re.finditer('{}'.format('\[([0-9]+)\]'*i),string))
    return (0,0) if not i else (i,it) if len(it) else dimensions(string,i-1)
def order(string):
    it=re.findall('\$O([0-9]+)',string)
    return int(it[0]) if len(it) else 0
def dictionary_order(R,L):
    return 1 if R>L else 0 if R==L else -1
def iter_bit(dim,R,L):
    for i in range(dim):
        if R[i]!=L[i]:
            return A(R[i],L[i])
    return 0
def dimension_based(R,L):
    rd=dimensions(R[0],2)
    ld=dimensions(L[0],2)
    if rd[0]==ld[0] and rd[0]!=0:
        X,Y=order(R[0]),order(L[0])
        if R[0][:rd[1][0].start()]==L[0][:ld[1][0].start()]:
            if X-1>=len(rd[1]) and Y-1>=len(ld[1]):
                return 1 if X>Y else -1
            if X-1>=len(rd[1]) or Y-1>=len(ld[1]):
                return 1 if X-1>=len(rd[1]) else -1
            res=iter_bit(rd[0],rd[1][X-1].groups(),ld[1][Y-1].groups())
            return res if res else 1 if X > Y else -1 
    return dictionary_order(R[0],L[0])
       
def order_key_method(origin_mapping):
    mmm = list(origin_mapping.items())
    mmm=sorted(mmm,key=cmp_to_key(dimension_based))
    a0,a1 =zip(*mmm)
    aaa = list(zip(a1,[i for i in range(len(mmm))]))
    aaa=sorted(aaa,key=cmp_to_key(dimension_based))
    aaaa = [*zip(*aaa)]
    return dict(zip(*[list(a0),aaaa[0]]))

def check_type(origin_mapping):
    output_script = ""
    choise = -1
    methods = ['Default', 'Natural', 'Order']
    #methods = ['Default', 'Natural']
    functions = [default_key_method, natural_key_method, order_key_method]
    data = [ [ [] for j in range(4) ] for i in range(len(methods)) ] # index, one level run-length, readable, two level run-length
    for i in range(len(data)):
        if choise == -1:
            input_table = functions[i](origin_mapping)
            flag, data[i][0], data[i][1], data[i][2], data[i][3] = generate_tables(origin_mapping, input_table, methods[i])
            choise = i if flag == i else -1   
    if choise != -1:
        output_script = complete_match[choise]
    else:
        print('No method can solve this case.')
        two_level = [ len(data[i][3]) for i in range(len(methods)) ]
        t_index = two_level.index(min(two_level))
        print('Using %s key table to record the original relationship.' % methods[t_index])
        index, one_level, readable, two_level = data[t_index][0], data[t_index][1], data[t_index][2], data[t_index][3] 
        #print(two_level)
        #print(incomplete_match[t_index])
        output_script = incomplete_match[t_index].format("".join(str(two_level).split(' ')))
    print('Script size: %d.' % len(output_script))
    return output_script

def test_all(case):
    print('case: {}'.format(case))
    map_in_name, script_name = '../cases/case{}.json'.format(case), 'test.py'
    map_in_file = open(map_in_name)
    origin_mapping = json.load(map_in_file)
    map_in_file.close()  
    print('Script:\n%s\n'%check_type(origin_mapping))
    print()

if __name__ == '__main__':
    for i in range(0,9,1):
        test_all(i)
    


case: 0
Default key can sovle this case.
Script size: 105.
Script:
import json as a,sys
b=sys.argv
a.dump(dict(zip(*[sorted(x)for x in a.load(open(b[1]))])),open(b[2],'w'))


case: 1
Default key can't sovle this case.
Shift count in this method: 10442.
Shift table string length: 45962.
One level run-length table string length: 20453.
The most large consecutive segment length: 429.
Two level run-length table string length: 20935.
Natural key can't sovle this case.
Shift count in this method: 3274.
Shift table string length: 42299.
One level run-length table string length: 7518.
The most large consecutive segment length: 429.
Two level run-length table string length: 7362.
Order key can't sovle this case.
Shift count in this method: 2919.
Shift table string length: 41964.
One level run-length table string length: 5054.
The most large consecutive segment length: 8259.
Two level run-length table string length: 4708.
No method can solve this case.
Using Order key table to record the origina

Shift table string length: 189225.
One level run-length table string length: 1884.
The most large consecutive segment length: 59738.
Two level run-length table string length: 1914.
Natural key can sovle this case.
Script size: 179.
Script:
import json as a,sys,re
b=sys.argv
a.dump(dict(zip(*[sorted(i,key=lambda y:[int(c)if c.isdigit()else c for c in re.split('(\d+)',y)])for i in a.load(open(b[1]))])),open(b[2],'w'))


case: 4
Default key can't sovle this case.
Shift count in this method: 15.
Shift table string length: 101209.
One level run-length table string length: 34.
The most large consecutive segment length: 33394.
Two level run-length table string length: 34.
Natural key can sovle this case.
Script size: 179.
Script:
import json as a,sys,re
b=sys.argv
a.dump(dict(zip(*[sorted(i,key=lambda y:[int(c)if c.isdigit()else c for c in re.split('(\d+)',y)])for i in a.load(open(b[1]))])),open(b[2],'w'))


case: 5
Default key can't sovle this case.
Shift count in this method: 9435.
Shift ta

In [80]:
map_in_file = open('../cases/case{}.json'.format(5))
origin_mapping = json.load(map_in_file)
input_table = natural_key_method(origin_mapping)
flag, index, one_level, readable, two_level = generate_tables(origin_mapping, input_table, 'Natural')
# two level 
t=two_level
# flatten
f=lambda z:[x for y in z for x in y]


Natural key can't sovle this case.
Shift count in this method: 84.
Shift table string length: 166598.
One level run-length table string length: 537.
The most large consecutive segment length: 24028.
Two level run-length table string length: 531.


In [149]:
flatten = """f=lambda z:[x for y in z for x in y]
"""
test = """def A(rhs,lhs):
    return 1 if int(rhs)>int(lhs) else -1
def dimensions(string,i):
    it=list(re.finditer('{}'.format('\[([0-9]+)\]'*i),string))
    return (0,0) if not i else (i,it) if len(it) else dimensions(string,i-1)
def order(string):
    it=re.findall('\$O([0-9]+)',string)
    return int(it[0]) if len(it) else 0
def dictionary_order(R,L):
    return 1 if R>L else 0 if R==L else -1
def iter_bit(dim,R,L):
    for i in range(dim):
        if R[i]!=L[i]:
            return A(R[i],L[i])
    return 0
def dimension_based(R,L):
    rd=dimensions(R[0],2)
    ld=dimensions(L[0],2)
    if rd[0]==ld[0] and rd[0]!=0:
        X,Y=order(R[0]),order(L[0])
        if R[0][:rd[1][0].start()]==L[0][:ld[1][0].start()]:
            if X-1>=len(rd[1]) and Y-1>=len(ld[1]):
                return 1 if X>Y else -1
            if X-1>=len(rd[1]) or Y-1>=len(ld[1]):
                return 1 if X-1>=len(rd[1]) else -1
            res=iter_bit(rd[0],rd[1][X-1].groups(),ld[1][Y-1].groups())
            return res if res else 1 if X > Y else -1 
    return dictionary_order(R[0],L[0])"""
# two level flatten
R=f([f([[j,1]for j in t[i]])if type(t[i])==list else t[i:i+2]for i in range(0,len(t),2)])
# one level flatten
O=f([[R[i]for j in range(R[i+1])]for i in range(0,len(R),2)])
print('Flatten function script length: %d.' % len(test1))
print('test script length: %d.' % len(test))


Flatten function script length: 37.
test script length: 1078.


In [135]:
A=[list(i)for i in[*zip(*input_table.items())]]
B=A[1][:]
count = 0
cint = 0
clist = 0
print(t)
err = 0
for i in range(len(origin_mapping)):
    if origin_mapping[A[0][i]] != A[1][i]: 
        err += 1
print('err count: %d.' % err)
test = []
for i in range(0,len(t),2):
    if type(t[i]) != list:
        test += [B[t[i]+count+j] for j in range(t[i+1])]
        count += t[i+1]
    else:
        test += [B[t[i][j]+count+j]for j in range(len(t[i]))]
        count += len(t[i])
print(len(test))
A[1]=test
# for i in range(0,len(t),2):
#     if type(t[i]) != list:
#         cint += 1
#         print(t[i:i+2])
#         for j in range(t[i+1]):
#             if t[i] != 0:
#                 A[1][count+j] = B[t[i]+count+j]
#         count += t[i+1]
#     else:
#         clist += 1
#         count += len(t[i])

err = 0        
for i in range(len(origin_mapping)):
    if origin_mapping[A[0][i]] != A[1][i]: 
        err += 1
print('err count: %d.' % err)
print("cint: %d, clist: %d." % (cint,clist))

[0, 8536, 1, 20, [-20], 1, 0, 24028, [1, -1], 1, 0, 352, 1, 2, [-2], 1, 0, 5, 1, 2, [-2], 1, 0, 121, 1, 6, [-6], 1, 0, 418, 1, 2, [-2], 1, 0, 229, [3, -1, 2, -2, 1, -3], 1, 0, 22, 1, 2, [-2], 1, 0, 417, 1, 2, [-2], 1, 0, 328, [1, -1], 1, 0, 61, 1, 2, [-2], 1, 0, 83, 1, 2, [-2], 1, 0, 744, [1, -1], 1, 0, 91, 1, 2, [-2], 1, 0, 280, [1, -1], 1, 0, 23, [1, -1], 1, 0, 47, [1, -1], 1, 0, 209, [1, -1], 1, 0, 47, [1, -1], 1, 0, 4297, [1, -1], 1, 0, 7055, [1, -1], 1, 0, 388, [1, -1], 1, 0, 558, [1, -1], 1, 0, 6839, [1, -1], 1, 0, 260]
err count: 84.
55522
err count: 0.
cint: 0, clist: 0.


In [144]:
T_natural_key_method_document='''import json as a,sys,re
t={}
n,o=sys.argv[1:3]
A=[sorted(i,key=lambda y:[int(c)if c.isdigit()else c for c in re.split('(\d+)',y)])for i in a.load(open(n))]
B=A[1][:]
D=[]
C=0
for i in range(0,len(t),2):
    if type(t[i])!=list:
        D+=[B[t[i]+C+j]for j in range(t[i+1])]
        C+=t[i+1]
    else:
        D+=[B[t[i][j]+C+j]for j in range(len(t[i]))]
        C+=len(t[i])
A[1]=D
a.dump(dict(zip(*A)),open(o,'w'))'''.format(str(t))
print(len(str(t)))
print(len(T_natural_key_method_document))

531
946


In [174]:
t=[0, 8259, 1, 2, [3]]
tt='[0, 8259, 1, 2, [3]]'
test = "t={}".format(str(t).strip(' '))
print(test)
print(len(tt))
print(len(tt.strip(' ')))
ttt=str(t).split(' ')
print(len(''.join(ttt)))

t=[0, 8259, 1, 2, [3]]
20
20
16
